# 키워드 추출 모델 비교 및 테스트

이 노트북에서는 TF-IDF, YAKE, RAKE, KeyBERT 모델을 사용하여 키워드 추출을 수행하고 성능을 비교합니다.

## 1. 라이브러리 임포트

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict
import time
import warnings
warnings.filterwarnings('ignore')

# 커스텀 모듈 임포트
from data_loader import DataLoader
from tfidf_extractor import TFIDFKeywordExtractor
from yake_extractor import YAKEKeywordExtractor
from rake_extractor import RAKEKeywordExtractor
from keybert_extractor import KeyBERTKeywordExtractor

# 시각화 설정
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## 2. 데이터 로드 및 탐색

In [ ]:
# 데이터 로더 초기화
data_loader = DataLoader('./clean_data.csv')

print(f"전체 문서 수: {data_loader.get_total_documents():,}")

# 샘플 문서 확인
sample_doc = data_loader.get_document_by_index(0)
print(f"\n첫 번째 문서 (처음 200자):")
print(f"{sample_doc[:200]}...")

## 3. 키워드 추출기 초기화

In [ ]:
# 모든 키워드 추출기 초기화
print("키워드 추출기 초기화 중...")

extractors = {
    'TF-IDF': TFIDFKeywordExtractor(),
    'YAKE': YAKEKeywordExtractor(),
    'RAKE': RAKEKeywordExtractor(),
    'KeyBERT': KeyBERTKeywordExtractor()
}

print("초기화 완료!")

## 4. 단일 문서 키워드 추출 예제

In [ ]:
# 테스트용 문서 선택
test_doc_idx = 100
test_document = data_loader.get_document_by_index(test_doc_idx)

print(f"테스트 문서 (인덱스: {test_doc_idx}):")
print(f"{test_document[:300]}...")
print("\n" + "="*80)

# 각 모델별 키워드 추출
for model_name, extractor in extractors.items():
    print(f"\n[{model_name}]")
    start_time = time.time()
    
    try:
        keywords_with_scores = extractor.extract_keywords(test_document, 5)
        keywords_only = extractor.get_keywords_only(test_document, 5)
        end_time = time.time()
        
        print(f"키워드: {keywords_only}")
        print(f"처리 시간: {end_time - start_time:.3f}초")
        
        if keywords_with_scores:
            print("상세 점수:")
            for i, (keyword, score) in enumerate(keywords_with_scores, 1):
                print(f"  {i}. {keyword}: {score:.4f}")
                
    except Exception as e:
        print(f"오류 발생: {e}")

## 5. 다중 문서 성능 비교

In [ ]:
# 성능 테스트용 랜덤 문서 선택
n_test_docs = 20
test_docs = data_loader.get_random_documents(n_test_docs)

print(f"{n_test_docs}개 문서로 성능 테스트 시작...")

# 결과 저장용 딕셔너리
performance_results = {
    model_name: {
        'times': [],
        'success_count': 0,
        'total_keywords': 0,
        'keyword_examples': []
    } for model_name in extractors.keys()
}

# 각 문서에 대해 테스트 수행
for i, (doc_idx, document) in enumerate(test_docs):
    print(f"\r문서 {i+1}/{n_test_docs} 처리 중...", end='')
    
    for model_name, extractor in extractors.items():
        start_time = time.time()
        
        try:
            keywords = extractor.get_keywords_only(document, 5)
            end_time = time.time()
            
            processing_time = end_time - start_time
            performance_results[model_name]['times'].append(processing_time)
            
            if keywords:
                performance_results[model_name]['success_count'] += 1
                performance_results[model_name]['total_keywords'] += len(keywords)
                if len(performance_results[model_name]['keyword_examples']) < 3:
                    performance_results[model_name]['keyword_examples'].append(keywords)
                    
        except Exception as e:
            end_time = time.time()
            performance_results[model_name]['times'].append(end_time - start_time)

print("\n완료!")

## 6. 성능 결과 분석

In [ ]:
# 성능 통계 계산
performance_stats = {}

for model_name, results in performance_results.items():
    times = results['times']
    success_rate = results['success_count'] / n_test_docs * 100
    avg_keywords = results['total_keywords'] / results['success_count'] if results['success_count'] > 0 else 0
    
    performance_stats[model_name] = {
        'avg_time': np.mean(times),
        'median_time': np.median(times),
        'std_time': np.std(times),
        'success_rate': success_rate,
        'avg_keywords': avg_keywords
    }

# 결과 DataFrame 생성
stats_df = pd.DataFrame(performance_stats).T
print("성능 비교 결과:")
print(stats_df.round(4))

## 7. 시각화

In [ ]:
# 처리 시간 비교 박스플롯
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. 처리 시간 박스플롯
time_data = [performance_results[model]['times'] for model in extractors.keys()]
ax1.boxplot(time_data, labels=extractors.keys())
ax1.set_title('모델별 처리 시간 분포')
ax1.set_ylabel('처리 시간 (초)')
ax1.tick_params(axis='x', rotation=45)

# 2. 평균 처리 시간 막대 그래프
models = list(performance_stats.keys())
avg_times = [performance_stats[model]['avg_time'] for model in models]
bars = ax2.bar(models, avg_times, color=['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon'])
ax2.set_title('모델별 평균 처리 시간')
ax2.set_ylabel('평균 처리 시간 (초)')
ax2.tick_params(axis='x', rotation=45)

# 막대 위에 값 표시
for bar, time_val in zip(bars, avg_times):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
             f'{time_val:.3f}', ha='center', va='bottom')

# 3. 성공률 비교
success_rates = [performance_stats[model]['success_rate'] for model in models]
bars = ax3.bar(models, success_rates, color=['gold', 'lightblue', 'lightpink', 'lightsteelblue'])
ax3.set_title('모델별 성공률')
ax3.set_ylabel('성공률 (%)')
ax3.set_ylim(0, 105)
ax3.tick_params(axis='x', rotation=45)

# 막대 위에 값 표시
for bar, rate in zip(bars, success_rates):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{rate:.1f}%', ha='center', va='bottom')

# 4. 평균 키워드 수
avg_keywords = [performance_stats[model]['avg_keywords'] for model in models]
bars = ax4.bar(models, avg_keywords, color=['mediumpurple', 'mediumseagreen', 'tomato', 'orange'])
ax4.set_title('모델별 평균 키워드 수')
ax4.set_ylabel('평균 키워드 수')
ax4.tick_params(axis='x', rotation=45)

# 막대 위에 값 표시
for bar, kw_count in zip(bars, avg_keywords):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
             f'{kw_count:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 8. 키워드 예시 비교

In [ ]:
# 각 모델의 키워드 추출 예시 출력
print("모델별 키워드 추출 예시:")
print("=" * 80)

for model_name, results in performance_results.items():
    print(f"\n[{model_name}]")
    examples = results['keyword_examples']
    
    if examples:
        for i, keywords in enumerate(examples, 1):
            print(f"  예시 {i}: {keywords}")
    else:
        print("  추출된 예시가 없습니다.")

## 9. 모델 추천 및 결론

In [ ]:
# 모델 순위 계산
print("모델 성능 순위:")
print("=" * 50)

# 속도 순위 (빠른 순)
speed_ranking = sorted(performance_stats.items(), key=lambda x: x[1]['avg_time'])
print("\n속도 순위 (빠른 순):")
for i, (model, stats) in enumerate(speed_ranking, 1):
    print(f"  {i}. {model}: {stats['avg_time']:.3f}초")

# 성공률 순위 (높은 순)
success_ranking = sorted(performance_stats.items(), key=lambda x: x[1]['success_rate'], reverse=True)
print("\n성공률 순위 (높은 순):")
for i, (model, stats) in enumerate(success_ranking, 1):
    print(f"  {i}. {model}: {stats['success_rate']:.1f}%")

# 종합 추천
print("\n" + "=" * 50)
print("모델 추천:")
print("=" * 50)

fastest_model = speed_ranking[0][0]
most_reliable = success_ranking[0][0]

print(f"\n• 빠른 처리가 필요한 경우: {fastest_model}")
print(f"• 안정적인 결과가 필요한 경우: {most_reliable}")
print(f"• 고품질 키워드가 필요한 경우: KeyBERT (의미론적 분석)")
print(f"• 실시간 처리가 필요한 경우: RAKE 또는 YAKE")

print("\n결론:")
print("- TF-IDF: 안정적이고 빠른 전통적 방법")
print("- YAKE: 비지도 학습, 빠른 처리, 다국어 지원")
print("- RAKE: 가장 빠른 처리, 구문 정보 활용")
print("- KeyBERT: 최고 품질, 의미론적 유사성, 느린 처리")

## 10. 사용자 정의 테스트

In [ ]:
# 사용자가 직접 문서 인덱스를 입력하여 테스트
def test_custom_document(doc_index: int):
    """사용자 지정 문서 테스트 함수"""
    document = data_loader.get_document_by_index(doc_index)
    
    if document is None:
        print(f"문서 인덱스 {doc_index}를 찾을 수 없습니다.")
        return
    
    print(f"문서 {doc_index} 키워드 추출 결과:")
    print(f"문서 내용: {document[:200]}...")
    print("-" * 60)
    
    for model_name, extractor in extractors.items():
        try:
            keywords = extractor.get_keywords_only(document, 5)
            print(f"{model_name:8}: {keywords}")
        except Exception as e:
            print(f"{model_name:8}: 오류 - {e}")

# 예시 테스트 (원하는 인덱스로 변경 가능)
test_custom_document(500)

In [ ]:
# 사용자 정의 텍스트로 테스트
custom_text = """
Artificial intelligence and machine learning have revolutionized the way we process and analyze data.
Deep learning models, particularly neural networks, have shown remarkable performance in natural language processing,
computer vision, and speech recognition tasks. These technologies are now being applied in various domains
including healthcare, finance, and autonomous systems.
"""

print("사용자 정의 텍스트 키워드 추출:")
print(f"텍스트: {custom_text.strip()}")
print("-" * 60)

for model_name, extractor in extractors.items():
    try:
        keywords = extractor.get_keywords_only(custom_text, 5)
        print(f"{model_name:8}: {keywords}")
    except Exception as e:
        print(f"{model_name:8}: 오류 - {e}")